# Hubble Ultra Deep Field (f105w)

### reusing the sep workflow on the udf f105w image and then grabbing basic flux stats and a fake 3 color image towards the end

In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse


sep.set_extract_pixstack(1000000)     # allow more active pixels
sep.set_sub_object_limit(100000)      # allow more sub-objects when deblending

%matplotlib inline
rcParams["figure.figsize"] = [10., 8.]



In [ ]:
path = "../data/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_wht.fits"

img = fits.getdata(path)
img = img.astype(img.dtype.newbyteorder('='))

mid, spread = np.mean(img), np.std(img)

fig, ax = plt.subplots()
show = ax.imshow(
    img,
    interpolation = "nearest",
    cmap = "gray",
    vmin = mid - spread,
    vmax = mid + spread,
    origin = "lower",
)

fig.colorbar(show,ax=ax)
fig.suptitle("UDF f105w frame")
fig.savefig("../figs/udf105_img.png", dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
backg = sep.Background(img)
img_sub = img - backg

print("f105w backg level:", backg.globalback)
print("f105w backg rms:", backg.globalrms)

In [ ]:
sources105 = sep.extract(img_sub, 2.5, err=backg.globalrms)
len(sources105)

In [ ]:
fig, ax = plt.subplots()
midimg_sub, spreadimgsub = np.mean(img_sub), np.std(img_sub)

show = ax.imshow(
    img_sub,
    interpolation = "nearest",
    cmap = "gray",
    vmin = midimg_sub - spreadimgsub,
    vmax = midimg_sub + spreadimgsub,
    origin = "lower",
)

for i in range(len(sources105)):
    e = Ellipse(
        xy=(sources105["x"][i], sources105["y"][i]),
        width=6 * sources105["a"][i],
        height=6 * sources105["b"][i],
        angle=sources105["theta"][i] * 180.0 / math.pi,
    )
    e.set_facecolor("none")
    e.set_edgecolor("red")
    ax.add_artist(e)

fig.suptitle("Sources detected in f105w")
fig.savefig("../figs/udf105_sources.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:

# ok its just producing a blank image